In [1]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import XGBModel
from sklearn.metrics import mean_squared_error, mean_squared_log_error
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../../Data/Kaggle/StoreSales/train_darts.csv", parse_dates=["date"])

In [3]:
df.head()

,date,store_nbr,family,sales,onpromotion,typeholiday,dcoilwtico,city,state,typestores,cluster,day_of_week,day,month,year
0,2013-01-01,1,0,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
1,2013-01-01,1,1,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
2,2013-01-01,1,2,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
3,2013-01-01,1,3,0.0,0,3,93.14,18,12,3,13,2,1,1,2013
4,2013-01-01,1,4,0.0,0,3,93.14,18,12,3,13,2,1,1,2013


In [4]:
unique_combinations = df[['store_nbr', 'family']].drop_duplicates()

In [5]:
total_rmse = 0
total_rmsle = 0
num_models = 0

In [6]:
trained_models = {}

In [7]:
for index, row in unique_combinations.iterrows():
    store_nbr = row['store_nbr']
    family = row['family']
    
    print(f"\nTraining model for store {store_nbr} and family {family}...")
    
    df_subset = df[(df['store_nbr'] == store_nbr) & (df['family'] == family)]
    
    ts = TimeSeries.from_dataframe(df_subset, value_cols=["sales"])

    training_size = int(len(ts) - 30)
    
    train = ts[:training_size]
    val = ts[training_size:]

    model = XGBModel(lags=30)
    model.fit(train)

    trained_models[(store_nbr, family)] = model

    predictions = model.predict(n=len(val))

    val = val.pd_series().tolist()
    predictions = predictions.pd_series().tolist()

    predictions = np.maximum(predictions, 0)

    rmse = np.sqrt(mean_squared_error(val, predictions))
    rmsle = np.sqrt(mean_squared_error(np.log1p(val), np.log1p(predictions)))

    total_rmse += rmse
    total_rmsle += rmsle
    num_models += 1

    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle}")

    plt.figure(figsize=(12, 6))
    plt.plot(val, label='Actual')
    plt.plot(predictions, label='Predictions')
    plt.title(f"Model for store {store_nbr} and family {family}")
    plt.legend()
    plt.savefig(f"XGBoost/Monthly/Lags_M06.6_store{store_nbr}_family{family}.png")
    plt.close()


Training model for store 1 and family 0...
Root Mean Squared Error (RMSE): 3.0558632070338376
Root Mean Squared Logarithmic Error (RMSLE): 0.6530189211160767

Training model for store 1 and family 1...
Root Mean Squared Error (RMSE): 1.6604496714611397e-16
Root Mean Squared Logarithmic Error (RMSLE): 1.6604496714611394e-16

Training model for store 1 and family 2...
Root Mean Squared Error (RMSE): 2.1646568522114404
Root Mean Squared Logarithmic Error (RMSLE): 0.4622102885451572

Training model for store 1 and family 3...
Root Mean Squared Error (RMSE): 353.8324174878417
Root Mean Squared Logarithmic Error (RMSLE): 0.20679418973535849

Training model for store 1 and family 4...
Root Mean Squared Error (RMSE): 0.6212291944414212
Root Mean Squared Logarithmic Error (RMSLE): 0.42589320753821786

Training model for store 1 and family 5...
Root Mean Squared Error (RMSE): 75.06519809453684
Root Mean Squared Logarithmic Error (RMSLE): 0.2581397405213897

Training model for store 1 and family

Root Mean Squared Error (RMSE): 38.838500692672525
Root Mean Squared Logarithmic Error (RMSLE): 0.22258659265078948

Training model for store 10 and family 19...
Root Mean Squared Error (RMSE): 3.201695494430173
Root Mean Squared Logarithmic Error (RMSLE): 0.7640114130935145

Training model for store 10 and family 20...
Root Mean Squared Error (RMSE): 0.0026159000117331743
Root Mean Squared Logarithmic Error (RMSLE): 0.002612484500424887

Training model for store 10 and family 21...
Root Mean Squared Error (RMSE): 1.9883445403202753
Root Mean Squared Logarithmic Error (RMSLE): 0.748207933283557

Training model for store 10 and family 22...
Root Mean Squared Error (RMSE): 9.542412184912036
Root Mean Squared Logarithmic Error (RMSLE): 0.6052905438412809

Training model for store 10 and family 23...
Root Mean Squared Error (RMSE): 0.875875072665245
Root Mean Squared Logarithmic Error (RMSLE): 0.547321845760703

Training model for store 10 and family 24...
Root Mean Squared Error (RMSE): 5

Root Mean Squared Error (RMSE): 32.96070368395703
Root Mean Squared Logarithmic Error (RMSLE): 0.17338970333931072

Training model for store 12 and family 6...
Root Mean Squared Error (RMSE): 12.452138858389242
Root Mean Squared Logarithmic Error (RMSLE): 0.7210858514404972

Training model for store 12 and family 7...
Root Mean Squared Error (RMSE): 862.680680740641
Root Mean Squared Logarithmic Error (RMSLE): 0.5851049432022112

Training model for store 12 and family 8...
Root Mean Squared Error (RMSE): 80.12084911346591
Root Mean Squared Logarithmic Error (RMSLE): 0.1892372407115291

Training model for store 12 and family 9...
Root Mean Squared Error (RMSE): 44.85676423514744
Root Mean Squared Logarithmic Error (RMSLE): 0.20887729956796244

Training model for store 12 and family 10...
Root Mean Squared Error (RMSE): 45.641550798309055
Root Mean Squared Logarithmic Error (RMSLE): 0.6622189748068413

Training model for store 12 and family 11...
Root Mean Squared Error (RMSE): 14.013004

Root Mean Squared Error (RMSE): 1.8363408438753939
Root Mean Squared Logarithmic Error (RMSLE): 0.6869515578593496

Training model for store 13 and family 24...
Root Mean Squared Error (RMSE): 33.353449812110746
Root Mean Squared Logarithmic Error (RMSLE): 0.24596614019120427

Training model for store 13 and family 25...
Root Mean Squared Error (RMSE): 45.77952052513738
Root Mean Squared Logarithmic Error (RMSLE): 0.24435169282678865

Training model for store 13 and family 26...
Root Mean Squared Error (RMSE): 1.1550650954358752
Root Mean Squared Logarithmic Error (RMSLE): 0.657663006692419

Training model for store 13 and family 27...
Root Mean Squared Error (RMSE): 5.666744200173567
Root Mean Squared Logarithmic Error (RMSLE): 0.7219751676600472

Training model for store 13 and family 28...
Root Mean Squared Error (RMSE): 31.27205326858645
Root Mean Squared Logarithmic Error (RMSLE): 0.24045652748099547

Training model for store 13 and family 29...
Root Mean Squared Error (RMSE): 9.6

Root Mean Squared Error (RMSE): 46.00505284590141
Root Mean Squared Logarithmic Error (RMSLE): 0.1441227883397023

Training model for store 15 and family 9...
Root Mean Squared Error (RMSE): 55.43381973488078
Root Mean Squared Logarithmic Error (RMSLE): 0.22348544831475112

Training model for store 15 and family 10...
Root Mean Squared Error (RMSE): 29.227868578408955
Root Mean Squared Logarithmic Error (RMSLE): 0.29991121425029044

Training model for store 15 and family 11...
Root Mean Squared Error (RMSE): 14.421773892033888
Root Mean Squared Logarithmic Error (RMSLE): 0.2065155677149018

Training model for store 15 and family 12...
Root Mean Squared Error (RMSE): 355.5407484497007
Root Mean Squared Logarithmic Error (RMSLE): 0.13265996701413182

Training model for store 15 and family 13...
Root Mean Squared Error (RMSE): 7.545659162475609
Root Mean Squared Logarithmic Error (RMSLE): 0.7916484867022242

Training model for store 15 and family 14...
Root Mean Squared Error (RMSE): 1.72

...

Root Mean Squared Error (RMSE): 15.892452269666071
Root Mean Squared Logarithmic Error (RMSLE): 0.3564892041907821

Training model for store 51 and family 21...
Root Mean Squared Error (RMSE): 4.794107549283055
Root Mean Squared Logarithmic Error (RMSLE): 0.633698328007577

Training model for store 51 and family 22...
Root Mean Squared Error (RMSE): 90.92267762170871
Root Mean Squared Logarithmic Error (RMSLE): 0.4883834896215269

Training model for store 51 and family 23...
Root Mean Squared Error (RMSE): 5.39145434531709
Root Mean Squared Logarithmic Error (RMSLE): 0.43136253995921026

Training model for store 51 and family 24...
Root Mean Squared Error (RMSE): 87.48839541423375
Root Mean Squared Logarithmic Error (RMSLE): 0.12425919692239461

Training model for store 51 and family 25...
Root Mean Squared Error (RMSE): 77.50647449609244
Root Mean Squared Logarithmic Error (RMSLE): 0.25285491227800583

Training model for store 51 and family 26...
Root Mean Squared Error (RMSE): 8.3570

Root Mean Squared Error (RMSE): 4.707725214795698
Root Mean Squared Logarithmic Error (RMSLE): 0.5821664553990483

Training model for store 53 and family 7...
Root Mean Squared Error (RMSE): 203.19377463932506
Root Mean Squared Logarithmic Error (RMSLE): 0.1921099343483788

Training model for store 53 and family 8...
Root Mean Squared Error (RMSE): 85.65061537667816
Root Mean Squared Logarithmic Error (RMSLE): 0.15957754540885327

Training model for store 53 and family 9...
Root Mean Squared Error (RMSE): 55.31823903362294
Root Mean Squared Logarithmic Error (RMSLE): 0.2027179944940433

Training model for store 53 and family 10...
Root Mean Squared Error (RMSE): 12.640576840240112
Root Mean Squared Logarithmic Error (RMSLE): 0.13001366354048166

Training model for store 53 and family 11...
Root Mean Squared Error (RMSE): 47.1594370922974
Root Mean Squared Logarithmic Error (RMSLE): 0.3307907112398196

Training model for store 53 and family 12...
Root Mean Squared Error (RMSE): 430.6141

Root Mean Squared Error (RMSE): 12.61109233151244
Root Mean Squared Logarithmic Error (RMSLE): 0.25478181552669915

Training model for store 54 and family 25...
Root Mean Squared Error (RMSE): 37.72841368790104
Root Mean Squared Logarithmic Error (RMSLE): 0.1935586864956018

Training model for store 54 and family 26...
Root Mean Squared Error (RMSE): 0.4391822665379058
Root Mean Squared Logarithmic Error (RMSLE): 0.2641780776915529

Training model for store 54 and family 27...
Root Mean Squared Error (RMSE): 2.2903290664584794
Root Mean Squared Logarithmic Error (RMSLE): 0.6330972371783499

Training model for store 54 and family 28...
Root Mean Squared Error (RMSE): 20.827021872784698
Root Mean Squared Logarithmic Error (RMSLE): 0.31709636904553873

Training model for store 54 and family 29...
Root Mean Squared Error (RMSE): 38.76604017795776
Root Mean Squared Logarithmic Error (RMSLE): 0.44936745715592313

Training model for store 54 and family 30...
Root Mean Squared Error (RMSE): 95

Root Mean Squared Error (RMSE): 27.130351195563723
Root Mean Squared Logarithmic Error (RMSLE): 0.10053301904226632

Training model for store 7 and family 11...
Root Mean Squared Error (RMSE): 45.34429180168787
Root Mean Squared Logarithmic Error (RMSLE): 0.2004181728765756

Training model for store 7 and family 12...
Root Mean Squared Error (RMSE): 485.167930909644
Root Mean Squared Logarithmic Error (RMSLE): 0.12813812444521713

Training model for store 7 and family 13...
Root Mean Squared Error (RMSE): 11.651330930238915
Root Mean Squared Logarithmic Error (RMSLE): 0.31233623346378764

Training model for store 7 and family 14...
Root Mean Squared Error (RMSE): 2.8754315983691243
Root Mean Squared Logarithmic Error (RMSLE): 0.6245118547291083

Training model for store 7 and family 15...
Root Mean Squared Error (RMSE): 18.135355867886485
Root Mean Squared Logarithmic Error (RMSLE): 0.5897026176958654

Training model for store 7 and family 16...
Root Mean Squared Error (RMSE): 32.54087

Root Mean Squared Error (RMSE): 29.450677488736304
Root Mean Squared Logarithmic Error (RMSLE): 0.20962354342134495

Training model for store 8 and family 30...
Root Mean Squared Error (RMSE): 640.0958634310579
Root Mean Squared Logarithmic Error (RMSLE): 0.12547902449649956

Training model for store 8 and family 31...
Root Mean Squared Error (RMSE): 2.4227776756701034
Root Mean Squared Logarithmic Error (RMSLE): 0.6535482377209373

Training model for store 8 and family 32...
Root Mean Squared Error (RMSE): 13.728170773499171
Root Mean Squared Logarithmic Error (RMSLE): 0.3360606147804023

Training model for store 9 and family 0...
Root Mean Squared Error (RMSE): 6.165374567991787
Root Mean Squared Logarithmic Error (RMSLE): 0.6124441576612046

Training model for store 9 and family 1...
Root Mean Squared Error (RMSE): 0.9263661422837072
Root Mean Squared Logarithmic Error (RMSLE): 0.5219366902360032

Training model for store 9 and family 2...
Root Mean Squared Error (RMSE): 4.416481339

In [8]:
avg_rmse = total_rmse / num_models
avg_rmsle = total_rmsle / num_models

In [9]:
print(f"Average Root Mean Squared Error (RMSE) across all models: {avg_rmse}")
print(f"Average Root Mean Squared Logarithmic Error (RMSLE) across all models: {avg_rmsle}")

Average Root Mean Squared Error (RMSE) across all models: 108.1027618123939
Average Root Mean Squared Logarithmic Error (RMSLE) across all models: 0.4097264444737467
